# Лабораторная №1
## Задание
Для изготовление книжной полки требуется 3 листа древесной плиты размером 80x20 см. и 2 листа размером 65x25 см.

Мастерская может закупить не более 20 листов древесной плиты размером 100x100 см. по цене 400 рублей за лист. Книжная полка продаётся по цене 1000 рублей. Дополнительные затраты на вспомогательные материалы и оплату работы составляют 200 рублей на каждую книжную полку.

**Требуется:**

Определить план работы мастерской, обеспечивающий максимальную прибыль

# Решение
Рассмотрим способы разделение листа 100х100 см. на листы 80х20 и 65х20 см.

Всего существует 6 способов разделения с точностью до незначимых сдвигов разрезов и поворотов заготовки:

| **Способ** | **80х20 (длинная)** | **65х25 (короткая)** |
|:----------:|:---------:|:---------:|
|     _1_    |     0     |     5     |
|     _2_    |     1     |     4     |
|     _3_    |     2     |     3     |
|     _4_    |     3     |     2     |
|     _5_    |     4     |     1     |
|     _6_    |     6     |     0     |

$N$ - количество закупленных плит

$ 0 \leq N \leq 20 $

$400*N$ - затраты на покупку плит

$n_i$ - число закупленных плит, распиленных i-м способом

$ n_i \geq 0$

$\sum_{n=1}^{6} n_i = N$

a - количество длинных досок, b - количество коротких досок

$a_i$ - количество длинных досок получающихся при распиле i-м способом

$b_i$ - количество коротких досок получающихся при распиле i-м способом

$a = \sum_{n=1}^{6} n_i * a_i$

$b = \sum_{n=1}^{6} n_i * b_i$

P - общее количество собранных полок


$2a = 3b$

$I = P * (1000 - 200)$ - полученных доход

$\alpha = a - a\%3$

$\beta = b - b\%2$

In [5]:
# Опции распила
options = {
    0: (0, 5),
    1: (1, 4),
    2: (2, 3),
    3: (3, 2),
    4: (4, 1),
    5: (6, 0)
}

In [6]:
import itertools

max_plywoods = 20

data = []

for N in range(1, max_plywoods + 1):
    for combination in itertools.combinations_with_replacement(range(6), N):
        total_long = 0
        total_short = 0
        for option in combination:
            long, short = options[option]
            total_long += long
            total_short += short

        alpha = total_long - (total_long % 3)  # Длинные доски, кратные 3
        beta = total_short - (total_short % 2)  # Короткие доски, кратные 2
        P = min(alpha/3, beta/2)  # Общее количество собранных полок

        revenue = P * 800
        expenses = 400 * N
        profit = revenue - expenses

        data.append({'profit': profit, 'N': N, 'combination': combination})


In [7]:
sorted_data = sorted(data, key = lambda x: x['profit'], reverse=True)

In [ ]:
sorted_data[:20]

## Симплекс метод

In [1]:
from utils import *

In [2]:
import numpy as np


A = np.array(
    [
        [1,  1,  1, 1, 1, 1, 0, 1, 0, 0],
        [-6, -4, -3, -2, -1, 0, 3, 0, 1, 0],
        [0, -1 , -2, -3, -4, -5, 2, 0, 0, 1]
    ],
    dtype= np.float64
)
b = np.array(
    [
        [20],
        [0],
        [0]
    ],
    dtype= np.float64
    )

F = np.array([400, 400, 400, 400, 400, 400, -800, 0, 0, 0], dtype= np.float64)
F = np.expand_dims(F, 0)

print(A)
print(b)
print(F)

[[ 1.  1.  1.  1.  1.  1.  0.  1.  0.  0.]
 [-6. -4. -3. -2. -1.  0.  3.  0.  1.  0.]
 [ 0. -1. -2. -3. -4. -5.  2.  0.  0.  1.]]
[[20.]
 [ 0.]
 [ 0.]]
[[ 400.  400.  400.  400.  400.  400. -800.    0.    0.    0.]]


In [3]:
m = np.zeros((3,1), dtype= np.float64)

In [4]:
def positive_zeros(A,b, m):
    A = np.where(A == -0, 0, A)
    b = np.where(b == -0, 0, b)
    m = np.where(m == -0, 0, m) 
    return A,b,m

In [5]:
for _ in range(100):
    if check_for_optimal_solution(F):
        break
    A,b,m = positive_zeros(A,b,m)
    for row in range(A.shape[0]):
        for col in range(A.shape[1]):
            if abs(A[row, col]) < 1e-6:
                A[row, col] = 0

    A,b,m = positive_zeros(A,b,m)

    leading_column = detect_leading_column(F)
    m = calculate_min(A, b, leading_column)
    A,b,m = positive_zeros(A,b,m)
    pivot_row = find_pivot_row(A, m, leading_column)
    
    A[pivot_row, :] = A[pivot_row, :] / A[pivot_row, leading_column]
    A,b,m = positive_zeros(A,b,m)
    print(A)
    print(b)
    print(f"{pivot_row=}")
    print(f"{leading_column=}")
    print(m)
    print('---')

    
    for row in range(A.shape[0]):
        if row != pivot_row:
            A[row, :] =  A[row, :] - A[pivot_row, :] * (A[row, leading_column] / A[pivot_row, leading_column])
    A,b,m = positive_zeros(A,b,m)
    F = F - A[pivot_row, :] * (F[0, leading_column] / A[pivot_row, leading_column])
    A,b,m = positive_zeros(A,b,m)
    for row in range(b.shape[0]):
        if row != pivot_row:
            b[row, :] =  b[row, :] - b[pivot_row, :] * (A[row, leading_column] / A[pivot_row, leading_column])
    A,b,m = positive_zeros(A,b,m)

[[ 1.          1.          1.          1.          1.          1.
   0.          1.          0.          0.        ]
 [-2.         -1.33333333 -1.         -0.66666667 -0.33333333  0.
   1.          0.          0.33333333  0.        ]
 [ 0.         -1.         -2.         -3.         -4.         -5.
   2.          0.          0.          1.        ]]
[[20.]
 [ 0.]
 [ 0.]]
pivot_row=1
leading_column=6
[[nan]
 [ 0.]
 [ 0.]]
---
[[ 1.          1.          1.          1.          1.          1.
   0.          1.          0.          0.        ]
 [-2.         -1.33333333 -1.         -0.66666667 -0.33333333  0.
   1.          0.          0.33333333  0.        ]
 [ 1.          0.41666667  0.         -0.41666667 -0.83333333 -1.25
   0.          0.         -0.16666667  0.25      ]]
[[20.]
 [ 0.]
 [ 0.]]
pivot_row=2
leading_column=0
[[20.]
 [ 0.]
 [ 0.]]
---
[[ 0.          0.58333333  1.          1.41666667  1.83333333  2.25
   0.          1.          0.16666667 -0.25      ]
 [ 0.         -0.5   

[[ 0.          0.58333333  1.          1.41666667  1.83333333  2.25
   0.          1.          0.16666667 -0.25      ]
 [ 0.         -0.5        -1.         -1.5        -2.         -2.5
   1.          0.          0.          0.5       ]
 [-0.8        -0.33333333  0.          0.33333333  0.66666667  1.
   0.          0.          0.13333333 -0.2       ]]
[[20.]
 [ 0.]
 [ 0.]]
pivot_row=2
leading_column=5
[[8.88888889]
 [0.        ]
 [0.        ]]
---
[[ 1.8         1.33333333  1.          0.66666667  0.33333333  0.
   0.          1.         -0.13333333  0.2       ]
 [-2.         -1.33333333 -1.         -0.66666667 -0.33333333  0.
   1.          0.          0.33333333  0.        ]
 [ 1.          0.41666667  0.         -0.41666667 -0.83333333 -1.25
   0.          0.         -0.16666667  0.25      ]]
[[20.]
 [ 0.]
 [ 0.]]
pivot_row=2
leading_column=0
[[11.11111111]
 [ 0.        ]
 [ 0.        ]]
---
[[ 0.          0.58333333  1.          1.41666667  1.83333333  2.25
   0.          1.       

In [ ]:
F

In [ ]:
-5 < - 1e-6

In [ ]:

b